# Example Audit Process and Documentation in Python
zyx client
<br>4/30/2018

This workpaper represents the testing of controls around user access for zyx.

## Risk
The user IDs of terminated employees are not removed promptly, which may result in unauthorized access to enterprise information.

# Control
HR flags personnel as Terminated ("T") in the user access tables when termination is processed.

# Procedures performed
1. We obtained a user listing as of 4/2/18 (48 hours after terminated employees report)
2. Obtained a listing of terminated employees as of 3/31/18
3. Compare terminated employees to user list to determine if any are listed as Active

### Load User List and Terminated List

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

In [2]:
users_file_path = 'files/users.xlsx'

In [3]:
users_df = pd.read_excel(users_file_path)

In [4]:
users_df

,UNIT_ID,USER_NAME,USER_ID,EMAIL_ADDR,ROLE_NAME,OBJ_NAME,FUNCTION_NAME,STATUS
0,11703,KARI ALEXANDER,KALEXANDER,kalexander@zyx.com,CONTRACTING OFFICER,Contract,View,A
1,20280,LINDA PARKER,LPARKER,lparker@zyx.com,NO LOGIN RIGHTS,Contract,View,A
2,30126,DANIEL YORK,DYORK,dyork@zyx.com,NO LOGIN RIGHTS,Contract,View,A
3,30187,CHARLES ALLEN,CALLEN,callen@zyx.com,NO LOGIN RIGHTS,Contract,View,A
4,30323,JAMIE MARTIN,JMARTIN,jmartin@zyx.com,ACCEPTANCE,Contract,View,A
5,30323,JAMIE MARTIN,JMARTIN,jmartin@zyx.com,SG_GENERAL ADMIN,Contract,View,A
6,30439,MICHAEL DAVIS,MDAVIS,mdavis@zyx.com,NO LOGIN RIGHTS,Contract,View,A
7,30508,JIMMY ALVAREZ,JALVAREZ,jalvarez@zyx.com,NO LOGIN RIGHTS,Contract,View,A
8,30630,MISS NICOLE YOUNG,MNICOLE,mnicole@zyx.com,NO LOGIN RIGHTS,Contract,View,A
9,31010,PAUL BREWER,PBREWER,pbrewer@zyx.com,NO LOGIN RIGHTS,Contract,View,A


In [5]:
term_path = 'files/term_employees_v1.xlsx'

In [6]:
term_df = pd.read_excel(term_path)

In [7]:
term_df

,Terminated Employees
0,MMEDINA
1,BGARRISON
2,KROGERS
3,MNEWMAN
4,EDAVIS
5,SSCHMIDT
6,DFLORES
7,JDALTON
8,JCAMPBELL
9,EHODGE


In [8]:
users_test = users_df[users_df['USER_ID'].isin(term_df['Terminated Employees'])]

In [9]:
users_test['TEST'] = users_test['STATUS'].map(lambda x: {'T': True, 'A': False}.get(x))

In [10]:
results = users_test[['USER_ID', 'STATUS', 'TEST']].groupby(['USER_ID', 'STATUS', 'TEST']).count().reset_index()

In [11]:
def summarize_results(results):
    number_of_terms = len(results)
    true_results = len(results[results['TEST']])
    false_results = len(results[~results['TEST']])
    print('----------------------RESULTS---------------------')
    print(f' There were {number_of_terms} terminations.')
    print(f' There were {true_results} users correctly shown as terminated.')
    if false_results:
        if false_results == 1:
            print(f' There was {false_results} user incorrectly shown as active.')
        else:
            print(f' There were {false_results} users incorrectly shown as active.')
    print('-'*50)

In [12]:
results

,USER_ID,STATUS,TEST
0,BGARRISON,T,True
1,DFLORES,T,True
2,EDAVIS,T,True
3,EHODGE,A,False
4,JCAMPBELL,T,True
5,JDALTON,T,True
6,KROGERS,T,True
7,MMEDINA,T,True
8,MNEWMAN,T,True
9,SSCHMIDT,T,True


In [13]:
summarize_results(results)

----------------------RESULTS---------------------
 There were 10 terminations.
 There were 9 users correctly shown as terminated.
 There was 1 user incorrectly shown as active.
--------------------------------------------------


# Conclusion
Controls are not operating effectively for termination flags.

In [14]:
!jupyter nbconvert --to html "full example.ipynb"

[NbConvertApp] Converting notebook full example.ipynb to html
[NbConvertApp] Writing 280495 bytes to full example.html
